In [14]:
%matplotlib inline
import math
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import xgboost as xgb

In [15]:
xytte = pd.read_csv("data/xytte.csv")

In [16]:
math.isnan(xytte["returned"][50548])

True

In [19]:
xytte_test = xytte[not math.isnan(xytte["returned"][50548])]

ValueError: Item wrong length 1 instead of 50578.

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 11)

N_train, _ = X_train.shape 
N_test,  _ = X_test.shape 
print N_train, N_test

In [8]:
gbm = xgb.XGBClassifier(silent=False, nthread=4, max_depth=10, n_estimators=800, subsample=0.5, learning_rate=0.03, seed=1337)
gbm.fit(train, target)
bst = gbm.booster()
imps = bst.get_fscore()

NameError: name 'train' is not defined